In [ ]:
import os 
import sys
import random
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json
import pydicom
from imgaug import augmenters as iaa
from tqdm import tqdm
import pandas as pd 
import glob
import keras
import tensorflow
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as prf


In [ ]:
print('keras:', keras.__version__)
print('tensorflow:',tensorflow.__version__)
!python3 --version

In [ ]:
import warnings 
warnings.filterwarnings("ignore")

In [ ]:
DATA_DIR = '/kaggle/input'

# Directory to save logs and trained model
ROOT_DIR = '/kaggle/working'

In [ ]:
!git clone https://www.github.com/matterport/Mask_RCNN.git
os.chdir('Mask_RCNN')
#!python setup.py -q install

In [ ]:
# Import Mask RCNN
sys.path.append(os.path.join(ROOT_DIR, 'Mask_RCNN'))  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

In [ ]:
train_dicom_dir = os.path.join(DATA_DIR, 'rsna-pneumonia-detection-challenge/stage_2_train_images')
test_dicom_dir = os.path.join(DATA_DIR, 'rsna-pneumonia-detection-challenge/stage_2_test_images')

In [ ]:
### Download COCO pre-trained weights
!wget --quiet https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5
!ls -lh mask_rcnn_coco.h5

COCO_WEIGHTS_PATH = "/kaggle/input/mask-rcnn-coco/mask_rcnn_coco.h5"

In [ ]:
def get_dicom_fps(dicom_dir):
    dicom_fps = glob.glob(dicom_dir+'/'+'*.dcm')
    return list(set(dicom_fps))

def parse_dataset(dicom_dir, anns):
    image_fps = get_dicom_fps(dicom_dir)
    image_annotations = {fp: [] for fp in image_fps}
    for index, row in anns.iterrows(): 
        fp = os.path.join(dicom_dir, row['patientId']+'.dcm')
        image_annotations[fp].append(row)
    return image_fps, image_annotations 

In [ ]:
hyper_paramters_comb={
    'backbone':['resnet50'],
    'learning_rate':[0.005],
    'batch_size':[8],
    'epochs':[10],
    'det_min_conf':[0.9],
    'det_nms_th':[0.8],
    'rpn_nms_th':[0.7],
    'steps_per_epoch':[135],
    'layers': ['heads']
}

hpc=pd.DataFrame(hyper_paramters_comb)

hpc['learning_rate'] = hpc['learning_rate'].astype(np.float32)
hpc['det_min_conf'] = hpc['det_min_conf'].astype(np.float32)
hpc['det_nms_th'] = hpc['det_nms_th'].astype(np.float32)
hpc['rpn_nms_th'] = hpc['rpn_nms_th'].astype(np.float32)

hpc.head()

In [ ]:
# training dataset
anns = pd.read_csv(os.path.join(DATA_DIR, 'rsna-pneumonia-detection-challenge/stage_2_train_labels.csv'))
anns.head()

In [ ]:
image_fps, image_annotations = parse_dataset(train_dicom_dir, anns=anns)

In [ ]:
ds = pydicom.read_file(image_fps[0]) # read dicom image from filepath 
image = ds.pixel_array # get image array

In [ ]:
# Original DICOM image size: 1024 x 1024
ORIG_SIZE = 1024

**Split the data into training and validation datasets that exctly macthes the dataset used for other models in the project**

In [ ]:
def dataset_model(image_names):
    org_train, org_val = train_test_split(anns, test_size=0.30, random_state=32)
    
    #  taking subset of train and validation dataset
    org_train_1=org_train[org_train.Target==1]
    org_train_0=org_train[org_train.Target==0]
    image_fps_train=org_train_1.patientId[:1500].tolist() + org_train_0.patientId[:500].tolist()
    image_fps_train=[train_dicom_dir+'/'+x+'.dcm' for x in image_fps_train]

    org_val_1=org_val[org_val.Target==1]
    org_val_0=org_val[org_val.Target==0]
    image_fps_val=org_val_1.patientId[:350].tolist() + org_val_0.patientId[:150].tolist()
    image_fps_val=[train_dicom_dir+'/'+x+'.dcm' for x in image_fps_val]
    
    image_fps_test=org_val_1.patientId[1500:1800].tolist() + org_val_0.patientId[500:700].tolist()
    image_fps_test=[train_dicom_dir+'/'+x+'.dcm' for x in image_fps_test]
    print(len(image_fps_train), len(image_fps_val), len(image_fps_test))

    return image_fps_train, image_fps_val, image_fps_test


In [ ]:
image_fps_train, image_fps_val, image_fps_test=dataset_model(image_fps)

In [ ]:
# c=math.ceil(len(image_fps)*0.7)
# image_fps_train,image_fps_val=image_fps[:c],image_fps[c:]
# image_fps_train, image_fps_test=image_fps_train[:2000], image_fps_train[2000:2500]
# image_fps_val=image_fps_val[:500]


In [ ]:
len(image_fps_train), len(image_fps_val), len(image_fps_test)

In [ ]:
class DetectorDataset(utils.Dataset):
    """Dataset class for training pneumonia detection on the RSNA pneumonia dataset.
    """

    def __init__(self, image_fps, image_annotations, orig_height, orig_width):
        super().__init__(self)
        
        # Add classes
        self.add_class('pneumonia', 1, 'Lung Opacity')
        
        # add images 
        for i, fp in enumerate(image_fps):
            annotations = image_annotations[fp]
            self.add_image('pneumonia', image_id=i, path=fp, 
                           annotations=annotations, orig_height=orig_height, orig_width=orig_width)
            
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

    def load_image(self, image_id):
        info = self.image_info[image_id]
        fp = info['path']
        ds = pydicom.read_file(fp)
        image = ds.pixel_array
        # If grayscale. Convert to RGB for consistency.
        if len(image.shape) != 3 or image.shape[2] != 3:
            image = np.stack((image,) * 3, -1)
        return image

    def load_mask(self, image_id):
        info = self.image_info[image_id]
        annotations = info['annotations']
        count = len(annotations)
        if count == 0:
            mask = np.zeros((info['orig_height'], info['orig_width'], 1), dtype=np.uint8)
            class_ids = np.zeros((1,), dtype=np.int32)
        else:
            mask = np.zeros((info['orig_height'], info['orig_width'], count), dtype=np.uint8)
            class_ids = np.zeros((count,), dtype=np.int32)
            for i, a in enumerate(annotations):
                if a['Target'] == 1:
                    x = int(a['x'])
                    y = int(a['y'])
                    w = int(a['width'])
                    h = int(a['height'])
                    mask_instance = mask[:, :, i].copy()
                    cv2.rectangle(mask_instance, (x, y), (x+w, y+h), 255, -1)
                    mask[:, :, i] = mask_instance
                    class_ids[i] = 1
        return mask.astype(np.bool), class_ids.astype(np.int32)

In [ ]:
# prepare the training dataset
dataset_train = DetectorDataset(image_fps_train, image_annotations, ORIG_SIZE, ORIG_SIZE)
dataset_train.prepare()

In [ ]:
# prepare the validation dataset
dataset_val = DetectorDataset(image_fps_val, image_annotations, ORIG_SIZE, ORIG_SIZE)
dataset_val.prepare()

**Display a random image with bounding boxes**

In [ ]:
# Load and display a random sample and their bounding boxes

class_ids = [0]
while class_ids[0] == 0:  ## look for a mask
    image_id = random.choice(dataset_train.image_ids)
    image_fp = dataset_train.image_reference(image_id)
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)

print(image.shape)

plt.figure(figsize=(10, 10))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.axis('off')

plt.subplot(1, 2, 2)
masked = np.zeros(image.shape[:2])
for i in range(mask.shape[2]):
    masked += image[:, :, 0] * mask[:, :, i]
plt.imshow(masked, cmap='gray')
plt.axis('off')


print(image_fp)
print(class_ids)

In [ ]:
# Image augmentation (light but constant)
augmentation = iaa.Sequential([
    iaa.OneOf([ ## geometric transform
        iaa.Affine(
            scale={"x": (0.98, 1.02), "y": (0.98, 1.04)},
            translate_percent={"x": (-0.02, 0.02), "y": (-0.04, 0.04)},
            rotate=(-2, 2),
            shear=(-1, 1),
        ),
        iaa.PiecewiseAffine(scale=(0.001, 0.025)),
    ]),
    iaa.OneOf([ ## brightness or contrast
        iaa.Multiply((0.9, 1.1)),
        iaa.ContrastNormalization((0.9, 1.1)),
    ]),
    iaa.OneOf([ ## blur or sharpen
        iaa.GaussianBlur(sigma=(0.0, 0.1)),
        iaa.Sharpen(alpha=(0.0, 0.1)),
    ]),
])

# test on the same image as above
imggrid = augmentation.draw_grid(image[:, :, 0], cols=5, rows=2)
plt.figure(figsize=(30, 12))
_ = plt.imshow(imggrid[:, :, 0], cmap='gray')

In [ ]:
# The following parameters have been selected to reduce running time for demonstration purposes 
# These are not optimal 

class DetectorConfig(Config):
    """Configuration for training pneumonia detection on the RSNA pneumonia dataset.
    Overrides values in the base Config class.
    """
    
    # Give the configuration a recognizable name  
    NAME = 'pneumonia'
    
    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 8
    
    BACKBONE = hpc.iloc[0]['backbone']
    BATCH_SIZE=hpc.iloc[0]['batch_size']
    NUM_CLASSES = 2  # background + 1 pneumonia classes
    
    IMAGE_MIN_DIM = 256
    IMAGE_MAX_DIM = 256
    RPN_ANCHOR_SCALES = (32, 64, 128, 256)
    TRAIN_ROIS_PER_IMAGE = 32
    MAX_GT_INSTANCES = 3
    DETECTION_MAX_INSTANCES = 3
    DETECTION_MIN_CONFIDENCE = hpc.iloc[0]['det_min_conf']
    DETECTION_NMS_THRESHOLD = hpc.iloc[0]['det_nms_th']
    RPN_NMS_THRESHOLD = hpc.iloc[0]['rpn_nms_th']
    STEPS_PER_EPOCH = hpc.iloc[0]['steps_per_epoch']


In [ ]:
config = DetectorConfig()
config.display()

In [ ]:
model = modellib.MaskRCNN(mode='training', config=config, model_dir=ROOT_DIR)

# Exclude the last layers because they require a matching number of classes
model.load_weights(COCO_WEIGHTS_PATH, by_name=True, exclude=[
    "mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])

In [ ]:
%%time
checkpoint_path = os.path.join(ROOT_DIR, "mask_rcnn_{}_*epoch*.h5".format(config.NAME.lower()))
checkpoint_path = checkpoint_path.replace("*epoch*", "{epoch:04d}")
callbacks = [keras.callbacks.ModelCheckpoint(checkpoint_path,verbose=1, save_weights_only=True,period=1)]

model.train(dataset_train, dataset_val, 
            learning_rate=hpc.iloc[0]['learning_rate'], 
            epochs=hpc.iloc[0]['epochs'], 
            custom_callbacks=callbacks,
            layers=hpc.iloc[0]['layers'],
            augmentation=augmentation)

history = model.keras_model.history.history

In [ ]:
epochs = range(1,len(next(iter(history.values())))+1)
pd.DataFrame(history, index=epochs)

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(111)
plt.plot(epochs, history["loss"], label="Train loss")
plt.plot(epochs, history["val_loss"], label="Valid loss")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(111)
plt.plot(epochs, history["mrcnn_class_loss"], label="Train class ce")
plt.plot(epochs, history["val_mrcnn_class_loss"], label="Valid class ce")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(111)
plt.plot(epochs, history["mrcnn_bbox_loss"], label="Train box loss")
plt.plot(epochs, history["val_mrcnn_bbox_loss"], label="Valid box loss")
plt.legend()
plt.show()

In [ ]:
best_epoch = np.argmin(history["val_loss"])
print("Best Epoch:", best_epoch + 1, history["val_loss"][best_epoch])

In [ ]:
x=filter(lambda f : (f.startswith('mask_rcnn_pneumonia') and ((4-len(str(best_epoch+1)))*str(0)+str(best_epoch+1)) in f), os.listdir(model.model_dir))
model_path=model.model_dir+'/'+list(x)[0]
print('Found model at {}'.format(model_path))

In [ ]:
# # select trained model 
# dir_names = next(os.walk(model.model_dir))[1]
# key = config.NAME.lower()
# dir_names = filter(lambda f: f.startswith(key), dir_names)
# dir_names = sorted(dir_names)
    
# fps = []
# # Pick last directory
# for d in dir_names: 
#     dir_name = os.path.join(model.model_dir, d)
#     # Find the last checkpoint
#     checkpoints = next(os.walk(dir_name))[2]
#     checkpoints = filter(lambda f: f.startswith("mask_rcnn"), checkpoints)
#     checkpoints = sorted(checkpoints)
#     if not checkpoints:
#         print('No weight files in {}'.format(dir_name))
#     else:
#         checkpoint = os.path.join(dir_name, checkpoints[best_epoch])
#         fps.append(checkpoint)

# model_path = sorted(fps)[-1]
# print('Found model {}'.format(model_path))

In [ ]:
class InferenceConfig(DetectorConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode='inference', config=inference_config, model_dir=ROOT_DIR)

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
# set color for class
def get_colors_for_class_ids(class_ids):
    colors = []
    for class_id in class_ids:
        if class_id == 1:
            colors.append((.941, .204, .204))
    return colors

**How does the predicted box compared to the expected value? Let's use the validation dataset to check.**

In [ ]:
# prepare the validation dataset
dataset_test = DetectorDataset(image_fps_test, image_annotations, ORIG_SIZE, ORIG_SIZE)
dataset_test.prepare()

In [ ]:
images=dataset_test.image_ids

ytrue=[]
ypred=[]

for i in range(len(images)):
        original_image,_, _, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset_test, inference_config, images[i], use_mini_mask=False)
        ytrue.append(gt_bbox)
        results=model.detect([original_image])
        ypred.append(results[0]['rois'])

In [ ]:
# Show few example of ground truth vs. predictions on the test dataset 
dataset = dataset_test
fig = plt.figure(figsize=(10, 30))
img_array=[]
for i in range(6):

    image_id = random.choice(dataset.image_ids)
    
    original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config, 
                               image_id, use_mini_mask=False)
    img_array.append(original_image)
    print(original_image.shape)
    plt.subplot(6, 2, 2*i + 1)
    visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                                dataset.class_names,
                                colors=get_colors_for_class_ids(gt_class_id), ax=fig.axes[-1])
    
    plt.subplot(6, 2, 2*i + 2)
    results = model.detect([original_image]) #, verbose=1)
    r = results[0]
    visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                                dataset.class_names, r['scores'], 
                                colors=get_colors_for_class_ids(r['class_ids']), ax=fig.axes[-1])


In [ ]:
def loadmasks(bboxes):
    mask = np.zeros((bboxes.shape[0], ORIG_SIZE, ORIG_SIZE))
    for i in range(bboxes.shape[0]):
        if bboxes[i]==[]:
            continue
        else:
            for bbox in bboxes[i]:
                x, y, w, h = bbox
                x1=math.floor(x)
                y1=math.floor(y)
                x2=math.ceil(x+w)
                y2=math.ceil(y+h)
                mask[i, y1:y2,x1:x2] = 1

    return mask

In [ ]:
def npmean_iou(bbox1, bbox2):
    mask1=loadmasks(np.array(bbox1))
    mask2=loadmasks(np.array(bbox2))
    union=np.count_nonzero(mask1, 1).astype(np.float32) + np.count_nonzero(mask2, 1).astype(np.float32)
    intersection = np.count_nonzero(np.logical_and(mask1, mask2), 1).astype(np.float32)
    smooth = np.ones(intersection.shape)
    iou = np.mean((intersection+smooth)/(union-intersection+smooth))
    return iou

iou=npmean_iou(ytrue, ypred)
iou

In [ ]:
prec, rec, f1s, _ = prf([1 if np.sum(x)>1 else 0 for x in ytrue], [1 if np.sum(x)>1 else 0 for x in ypred], average='binary')
prec, rec, f1s

In [ ]:
hpc.head()